In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Claude 3 Models
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/anthropic_claude_3_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/anthropic_claude_3_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>                                                                                               <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/anthropic_claude_3_intro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

### Claude on Vertex AI

Anthropic Claude 3 models on Vertex AI offer fully managed and serverless models are offered as managed APIs. To use a Claude model on Vertex AI, send a request directly to the Vertex AI API endpoint.

You can stream your Claude responses to reduce the end-user latency perception. A streamed response uses server-sent events (SSE) to incrementally stream the response.

### Available Anthropic Claude models

#### Claude 3 Sonnet
Anthropic Claude 3 Sonnet provides a balance between intelligence and speed for enterprise workloads. It's a high-endurance model for scaled AI that's available at a competitive price.

#### Claude 3 Haiku
Anthropic Claude 3 Haiku is the fastest, most compact model available from Anthropic. It is designed to answer simple queries and requests quickly. You can use it to build AI experiences that mimic human interactions.

All Claude 3 models can process images and return text outputs, and feature a 200K context window.

## Objective

This notebook shows how to use **Vertex AI API** and **Anthropic’s Vertex SDK for Python** to call the Claude on Vertex AI API with the Claude 3 Sonnet and Claude 3 Haiku model.

For more information, see the [Use Claude](https://cloud.devsite.corp.google.com/vertex-ai/generative-ai/docs/third-party-models/use-claude) documentation.


## Vertex AI API

### Getting Started


#### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, uncomment and run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

#### Select Claude 3 model

In [ ]:
MODEL = "claude-3-sonnet@20240229"  # @param ["claude-3-sonnet@20240229", "claude-3-haiku@20240307"]
if MODEL == "claude-3-sonnet@20240229":
    available_regions = ["us-central1", "asia-southeast1"]
else:
    available_regions = ["us-central1", "europe-west4"]

#### Select a region

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=available_regions,
    description="Select a region:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
    global REGION
    if change["type"] == "change" and change["name"] == "value":
        REGION = change.new
        print("Selected:", change.new)


REGION = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

#### Define Google Cloud project and model information

In [ ]:
# Initialize Vertex AI
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{REGION}-aiplatform.googleapis.com"

vertexai.init(project=PROJECT_ID, location=REGION)

#### Import required module

In [ ]:
import json

### Use the Claude 3.0 models with unary call

#### Text generation

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [{"role": "user", "content": "Send me a recipe for banana bread."}],
    "max_tokens": 100,
    "stream": False,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{REGION}/publishers/anthropic/models/{MODEL}:rawPredict -d '{request}'

#### Vision

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": "/9j/4AAQSkZJRgABAQEAZQBlAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAhADIDAREAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAABAUDBwACBggJ/8QAMRAAAQIFAgIJBAIDAAAAAAAAAQIDAAQFESEGEhMxBxQiMkFRYXGBCDM0chWxc5Gh/8QAGwEAAgMBAQEAAAAAAAAAAAAAAQIAAwQFBgf/xAAoEQACAQIFAwMFAAAAAAAAAAAAAQIDEQQSITFRBRNBBhQVFjJhcaH/2gAMAwEAAhEDEQA/AK46QZOnJQttIA2i4j5rh23ofZu+0rlaGrdUk1NBWAbCL3QzSHliklcSTsyublFqZRxF2Nh5xqo0nGSOLjsWpRdyrqlOT9OnVKmGFtLSrBIj0cIxlE8LOtOlUbQPPaxqMy2lDjpKUcswnt0aIdUnB6GM9IVSlUBKXlY9Ypl06Enex2KPqivTja4SOkiqkX4xzCfGwL/q2sXxqKt6jr7yn2KRPKbJthlUcqhhlTV2b62MWyEzOj9bV09VYoCwy7jetJS4g+YI/o3HpGmMYR1Zgq4ucvtDat0d1rQ9HYqWpZ95jqb7K+qhgrcmZdTmQ0QQFG/e57bi4FwTsoSjNOCW/nhnGxcpXVVy1XjlFxI6ANNa+0nTa9LNtqYqco1MtKFiClxN05+f94jN3Z0pNcFvap1op+GcdUfoll3QoySlXPgCYuWNn5KZYCm9jjal9EuoWVnguOW8M3EXLqD4KH05eGJFfSBrJKikKVYG3diz3y4K/jnyfRpvSFAbSGk0yXSD4bM2jk2sdrNc2XQKLS2Vzjkg20y2NyzsuQPYZJ8AALkmwiZQZkJqvpurahlnmOJJUqUWw5wZdxjfNbyggFxZuGBnIbSV2uN4MWRlkd7Fc1nVr6FLdB2mZmZ1HP6O11qgVH+Gprgo0hKqSWqSyp07XBsSkdYCVjKgVJBSARkG6vO7vFb/ANM2Hi03GT22/B6BoQmZmnMuTgSidauzNADsF1J2qUPIKtuA8AoCM1uDZfkZTEnxEi6m+eRaCgNsjNLYvya/5DC5g7rXYQkpcXsuUqwBe/jYXPteGEsRoZQ/Mdbm1vXSLtgm6Ri17ZF/U8swA/oBrWl6XWJZ1t5tpx5bS20ktrBuU2FlJOOZzBFEWmuhyj6P1NOaukKtPLnJySRTVpdG5KGUEFJHZuLWHMnAF72iPVWItJZjqJVXAccSHXgZjtK3NAjfa24kAZIAvASC3cmefaLfEbcmVFJ2kBmwP9+0NYW7By9n7JPrsESzCEeX7QAoNR+P8GCxWEMd8fpAIzJr7fwIgELpnn8QyGN5X8Ff+QQQEC++r3MQU//Z",
                    },
                },
                {"type": "text", "text": "What is in this image?"},
            ],
        }
    ],
    "max_tokens": 100,
    "stream": False,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{REGION}/publishers/anthropic/models/{MODEL}:rawPredict -d '{request}'

### Use the Claude 3.0 models with streaming call

#### Text Generation

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [{"role": "user", "content": "Send me a recipe for banana bread."}],
    "max_tokens": 100,
    "stream": True,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{REGION}/publishers/anthropic/models/{MODEL}:streamRawPredict -d '{request}'

#### Vision

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": "/9j/4AAQSkZJRgABAQEAZQBlAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQEBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAAhADIDAREAAhEBAxEB/8QAHAAAAgIDAQEAAAAAAAAAAAAABAUDBwACBggJ/8QAMRAAAQIFAgIJBAIDAAAAAAAAAQIDAAQFESEGEhMxBxQiMkFRYXGBCDM0chWxc5Gh/8QAGwEAAgMBAQEAAAAAAAAAAAAAAQIAAwQFBgf/xAAoEQACAQIFAwMFAAAAAAAAAAAAAQIDEQQSITFRBRNBBhQVFjJhcaH/2gAMAwEAAhEDEQA/AK46QZOnJQttIA2i4j5rh23ofZu+0rlaGrdUk1NBWAbCL3QzSHliklcSTsyublFqZRxF2Nh5xqo0nGSOLjsWpRdyrqlOT9OnVKmGFtLSrBIj0cIxlE8LOtOlUbQPPaxqMy2lDjpKUcswnt0aIdUnB6GM9IVSlUBKXlY9Ypl06Enex2KPqivTja4SOkiqkX4xzCfGwL/q2sXxqKt6jr7yn2KRPKbJthlUcqhhlTV2b62MWyEzOj9bV09VYoCwy7jetJS4g+YI/o3HpGmMYR1Zgq4ucvtDat0d1rQ9HYqWpZ95jqb7K+qhgrcmZdTmQ0QQFG/e57bi4FwTsoSjNOCW/nhnGxcpXVVy1XjlFxI6ANNa+0nTa9LNtqYqco1MtKFiClxN05+f94jN3Z0pNcFvap1op+GcdUfoll3QoySlXPgCYuWNn5KZYCm9jjal9EuoWVnguOW8M3EXLqD4KH05eGJFfSBrJKikKVYG3diz3y4K/jnyfRpvSFAbSGk0yXSD4bM2jk2sdrNc2XQKLS2Vzjkg20y2NyzsuQPYZJ8AALkmwiZQZkJqvpurahlnmOJJUqUWw5wZdxjfNbyggFxZuGBnIbSV2uN4MWRlkd7Fc1nVr6FLdB2mZmZ1HP6O11qgVH+Gprgo0hKqSWqSyp07XBsSkdYCVjKgVJBSARkG6vO7vFb/ANM2Hi03GT22/B6BoQmZmnMuTgSidauzNADsF1J2qUPIKtuA8AoCM1uDZfkZTEnxEi6m+eRaCgNsjNLYvya/5DC5g7rXYQkpcXsuUqwBe/jYXPteGEsRoZQ/Mdbm1vXSLtgm6Ri17ZF/U8swA/oBrWl6XWJZ1t5tpx5bS20ktrBuU2FlJOOZzBFEWmuhyj6P1NOaukKtPLnJySRTVpdG5KGUEFJHZuLWHMnAF72iPVWItJZjqJVXAccSHXgZjtK3NAjfa24kAZIAvASC3cmefaLfEbcmVFJ2kBmwP9+0NYW7By9n7JPrsESzCEeX7QAoNR+P8GCxWEMd8fpAIzJr7fwIgELpnn8QyGN5X8Ff+QQQEC++r3MQU//Z",
                    },
                },
                {"type": "text", "text": "What is in this image?"},
            ],
        }
    ],
    "max_tokens": 100,
    "stream": True,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{REGION}/publishers/anthropic/models/{MODEL}:streamRawPredict -d '{request}'

## Using Anthropic's Vertex SDK for *Python*

### Getting Started


#### Install Anthropic's Vertex SDK for Python

In [ ]:
! pip3 install -U 'anthropic[vertex]'

#### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Select Claude 3 model

In [ ]:
MODEL = "claude-3-sonnet@20240229"  # @param ["claude-3-sonnet@20240229", "claude-3-haiku@20240307"]
if MODEL == "claude-3-sonnet@20240229":
    available_regions = ["us-central1", "asia-southeast1"]
else:
    available_regions = ["us-central1", "europe-west4"]

#### Select a region

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=available_regions,
    description="Select a region:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
    global REGION
    if change["type"] == "change" and change["name"] == "value":
        REGION = change.new
        print("Selected:", change.new)


REGION = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

#### Define Google Cloud project and model information

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{REGION}-aiplatform.googleapis.com"

#### Authenticate your notebook environment
If you are running this notebook on Google Colab, uncomment run the following cell to authenticate your environment.

In [ ]:
# from google.colab import auth
# auth.authenticate_user(project_id=PROJECT_ID)

### Use the Claude 3.0 models with unary call

#### Text generation

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Send me a recipe for banana bread.",
        }
    ],
    model=MODEL,
)
print(message.model_dump_json(indent=2))

#### Vision

In [ ]:
import base64

import httpx
from anthropic import AnthropicVertex

client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)

image1_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image1_media_type = "image/jpeg"
image1_data = base64.b64encode(httpx.get(image1_url).content).decode("utf-8")

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image1_media_type,
                        "data": image1_data,
                    },
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ],
    model=MODEL,
)
print(message.model_dump_json(indent=2))

### Use the Claude 3.0 models with streaming call

#### Text generation

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region="us-central1", project_id=PROJECT_ID)

with client.messages.stream(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Send me a recipe for banana bread.",
        }
    ],
    model=MODEL,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

#### Vision

In [ ]:
import base64

import httpx
from anthropic import AnthropicVertex

image1_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image1_media_type = "image/jpeg"
image1_data = base64.b64encode(httpx.get(image1_url).content).decode("utf-8")
client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)

with client.messages.stream(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image1_media_type,
                        "data": image1_data,
                    },
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ],
    model=MODEL,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)